In [1]:
import pickle 
import numpy 
import sys     
sys.path.append("../") # add parent directory to path
import preprocessing # this project
import typing 

In [2]:
# just load data so functions can get test inputs 
# no need to go into final code 

[X, Y] = preprocessing.main(
    use_feature_categories=["socio-demographics", "health", "social", "neighbourhood"],
    csv_file="../ML_social.csv", dump=False)

/home/forrest/repos/AgeML/RNN/../preprocessing.py:196: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(csv_file)
100%|██████████| 26178/26178 [00:29<00:00, 875.60it/s]

(4, 18) (4, 4)


## prepare training data 

In [3]:
def slide_over_one_subject(subject_X, subject_Y, minimal_length):
    """Create samples from the data of one subject using a sliding window

    subject_X, subject_Y: 2D numpy array, rows for waves and columns for features
    subject_new_X, subject_new_Y: 3D numpy array, axis 0 for sliding window, axis 1 for waves, axis 2 for features

    """
    subject_new_X, subject_new_Y = [], []
    num_features = subject_X.shape[1]
    subject_X_windowed = numpy.lib.stride_tricks.sliding_window_view(
        subject_X, 
        window_shape=(minimal_length, num_features)
    ) 
    subject_new_X = subject_X_windowed[:, -1, :, :]

    num_targets = subject_Y.shape[1]
    subject_Y_windowed = numpy.lib.stride_tricks.sliding_window_view(
        subject_Y, 
        window_shape=(minimal_length, num_targets)
    ) 
    subject_new_Y = subject_Y_windowed[:, -1, :, :]

    return subject_new_X, subject_new_Y

def sampling_fixed_length(X, Y, minimal_length):
    new_X, new_Y = [], []
    for subject_X, subject_Y in zip(X, Y):
        if len(subject_Y)>= minimal_length:
            subject_new_X, subject_new_Y = slide_over_one_subject(subject_X, subject_Y, minimal_length=minimal_length)
            new_X += subject_new_X.tolist()
            new_Y += subject_new_Y.tolist()

    new_X, new_Y = numpy.array(new_X), numpy.array(new_Y)

    return new_X, new_Y 


In [4]:

# test the work on Y 
_, new_Y = sampling_fixed_length(X[:2], Y[:2], minimal_length=4) 
print (Y[:2])
print (new_Y)
print (new_Y[:, [0, -1], 1])
numpy.heaviside(new_Y[:, -1, 1]-new_Y[:, 0, 1], 0 )

[array([[0.44444444, 0.3       , 0.8       , 1.        ],
       [0.66666667, 0.6       , 0.8       , 1.        ],
       [0.40740741, 0.35      , 0.4       , 1.        ],
       [0.25925926, 0.2       , 0.2       , 1.        ]]), array([[0.66666667, 0.55      , 1.        , 1.        ],
       [0.59259259, 0.5       , 0.8       , 1.        ],
       [0.62962963, 0.5       , 1.        , 1.        ],
       [0.51851852, 0.4       , 0.8       , 1.        ],
       [0.62962963, 0.5       , 1.        , 1.        ]])]
[[[0.44444444 0.3        0.8        1.        ]
  [0.66666667 0.6        0.8        1.        ]
  [0.40740741 0.35       0.4        1.        ]
  [0.25925926 0.2        0.2        1.        ]]

 [[0.66666667 0.55       1.         1.        ]
  [0.59259259 0.5        0.8        1.        ]
  [0.62962963 0.5        1.         1.        ]
  [0.51851852 0.4        0.8        1.        ]]

 [[0.59259259 0.5        0.8        1.        ]
  [0.62962963 0.5        1.         1.        

array([0., 0., 0.])

In [5]:
def generate_labels(Y, target_index: int):
    """Extract the training label for a particular target dimension 

    Y: 3D ndarray, axis 0 is sample, axis 1 is wave, axis 2 is score 
    """
    # y_of_interest = Y[:, :, target_index] # 2D array 
    begin_score = Y[:, 0, target_index]
    end_score = Y[:, -1, target_index]

    labels = end_score - begin_score
    labels = numpy.heaviside(labels, 0)

    return labels.reshape((-1,1)) # 2d arrary, Nx1

# top level function for training data preparation 
# need to rerun for each target index rangeing from 0 to 2 
def prepare_training_data(X, Y, target_index: int, minimal_length):
    new_X, new_Y = sampling_fixed_length(X, Y, minimal_length=minimal_length) 
    new_Y = generate_labels(new_Y, target_index=target_index)
    return new_X, new_Y 


In [6]:
# Generate training data so we can test the model
# No need to go into final code 
train_X, train_y = prepare_training_data(X, Y, target_index=0, minimal_length=4)

## The networks

In [7]:
import torch.nn as nn
import torch

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
      super().__init__()
      self.rnn = torch.nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    #   self.fc1 = torch.nn.Linear(hidden_size, 3)
      self.fc1 = torch.nn.Linear(hidden_size, 1)

  def forward(self, x):
      output, hn = self.rnn(x)
      x = self.fc1(hn[0])
    #   x = self.fc2(x)
      x = torch.sigmoid(x)
      return x

class FC(nn.Module):
  def __init__(self, input_size):
      super().__init__()
      self.fc1 = torch.nn.Linear(input_size, 50)
      self.fc2 = torch.nn.Linear(50, 20)
      self.fc3 = torch.nn.Linear(20, 10)
      self.fc4 = torch.nn.Linear(10, 1)

  def forward(self, x):
      x = torch.flatten(x, start_dim=1) # first dimension is batch 
      x = self.fc1(x)
      x = torch.sigmoid(x)
      x = self.fc2(x)
      x = torch.sigmoid(x)
      x = self.fc3(x)
      x = torch.sigmoid(x)
      x = self.fc4(x)
      x = torch.sigmoid(x)
    
      return x

In [8]:
# test 
# USING VARIABLES PREVIOUSLY LOADED INTO THE MEMORY

train_X_tensor = torch.from_numpy(train_X.astype("float32"))
train_y_tensor = torch.from_numpy(train_y.astype("float32"))

input_dimension = train_X.shape[2]
net = RNN(input_dimension, 5, 2)
net(train_X_tensor[10:20]) 

tensor([[0.7076],
        [0.7131],
        [0.7114],
        [0.7085],
        [0.6459],
        [0.6243],
        [0.6957],
        [0.7096],
        [0.7107],
        [0.7075]], grad_fn=<SigmoidBackward0>)

# The trainer 

In [44]:
def train(net, X, y, 
          learning_rate, momentum, 
          batch_size, print_batch_size, 
          device: str 
          ):
    """

    net: a torch.nn instance
    X: 3D PyTorch Tensor, [sample, wave, feature]
    y: 2D PyTorch Tensor, Nx1, [sample, binary label]
    """

    if device != "cpu":
        net = net.to(torch.device(device))
        X = X.to(torch.device(device))
        y = y.to(torch.device(device))

    import torch.optim as optim

    criterion = nn.BCELoss(reduction="sum") 
    # criterion = nn.BCELoss(reduction="mean") 
    # criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    batch_size = batch_size
    print_batch_size = print_batch_size

    loss_log = [] 

    for epoch in range(3000):  # loop over the dataset multiple times

        running_loss = 0.0
        for i in range(0, X.shape[0], batch_size):

            optimizer.zero_grad()

            batch_X, batch_y = X[i:i+batch_size], y[i: i+batch_size]

            # forward + backward + optimize
            prediction = net.forward(batch_X)
            loss = criterion(prediction, batch_y)
            
            # update weights at the end of each batch
            loss.backward()
            optimizer.step() 

            # log loss
            loss_batch = loss.item()
            running_loss += loss_batch # accumulated loss of many batches until print 
            loss_log.append(loss_batch)

            # # print statistics
            # if i % print_batch_size == 0 and i / print_batch_size > 1:
            #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / (print_batch_size/batch_size):.7f}')
            #     running_loss = 0.0

        print (running_loss/X.shape[0])

    return loss_log

In [59]:
# Test the network training 
# USING VARIABLES PREVIOUSLY PLACE IN THE NOTEBOOK

input_dimension = train_X_tensor.shape[2]
net = RNN(input_dimension, 5, 2)

loss_log = train(net, train_X_tensor, train_y_tensor, 
                    learning_rate=0.001, 
                    momentum=0.2,
                    batch_size=128, print_batch_size=32*400, 
                    device="cuda:2")

0.6432653695655961
0.6414069774331123
0.6407597715887653
0.6403203783451685
0.6399142552742435
0.6394903282514903
0.6390434313074341
0.6386055558177229
0.6382209520268872
0.6379108667018187
0.6376706567181312
0.637485221074666
0.6373390095378644
0.6372192911751354
0.6371165740934297
0.6370241129817292
0.6369372270968013
0.6368527470305324
0.6367685899338402
0.6366835231598192
0.636597033613406
0.6365092590949533
0.636420870731179
0.6363329601998781
0.6362468129605912
0.6361637001728224
0.6360846933643151
0.6360105514932618
0.6359417051857653
0.6358782826546536
0.6358201714496389
0.6357671073450448
0.6357187178899179
0.6356745790394224
0.6356342544799415
0.6355973158603786
0.6355633803297537
0.6355320877871448
0.6355031289942602
0.6354762335504079
0.6354511637987293


KeyboardInterrupt: 

In [51]:
torch.cuda.get_device_name(2)

'NVIDIA GeForce RTX 2080 Ti'

In [ ]:
# shuffle the data and see whether first epoch is still the worst 


## Experiments

In [ ]:
def exp_one_target_one_arch(use_feature_categories: typing.List[str], csv_path:str, 
                            target_index:int, minimal_length:int, arch:str, 
                            learning_rate:float, momentum:float, 
                            batch_size:int, print_batch_size:int):

    [X, Y] = preprocessing.main(use_feature_categories= use_feature_categories,
                csv_file=csv_path, dump=False)    

    train_X, train_y = prepare_training_data(X, Y, 
                                target_index=target_index, 
                                minimal_length=4)

    # numpy float 64 for PyTorch float 32
    train_X = torch.from_numpy(train_X.astype("float32"))
    train_y = torch.from_numpy(train_y.astype("float32"))

    input_dimension = train_X.shape[2]
    if arch == "RNN":
        net = RNN(input_dimension, 5, 2)
    elif arch == "FC":
        net = FC(input_dimension)

    loss_log = train(net, train_X, train_y, 
                     learning_rate=learning_rate, 
                     momentum=momentum,
                     batch_size=batch_size, print_batch_size=print_batch_size)

In [12]:
# Test using one condition 

feature_combination = ["socio-demographics", "health", "social", "neighbourhood"]
CSV_raw = "../ML_social.csv"
    
loss_log = exp_one_target_one_arch(use_feature_categories=feature_combination, 
                        csv_path=CSV_raw,
                        target_index=0, minimal_length=4, 
                        arch="RNN", learning_rate=0.001, momentum=0.9,
                        batch_size=32, print_batch_size=32*10)

/home/forrest/repos/AgeML/RNN/../preprocessing.py:196: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(csv_file)
100%|██████████| 26178/26178 [00:30<00:00, 848.66it/s]


(4, 18) (4, 4)
[1,   641] loss: 1.4987940
[1,   961] loss: 0.6816145
[1,  1281] loss: 0.6550148
[1,  1601] loss: 0.6518805
[1,  1921] loss: 0.6256426
[1,  2241] loss: 0.6523841
[1,  2561] loss: 0.6342107
[1,  2881] loss: 0.6464762
[1,  3201] loss: 0.6457524
[1,  3521] loss: 0.6384335
[1,  3841] loss: 0.6361469
[1,  4161] loss: 0.6024196
[1,  4481] loss: 0.6382607
[1,  4801] loss: 0.6364564
[1,  5121] loss: 0.6012394
[1,  5441] loss: 0.6619647
[1,  5761] loss: 0.6275776
[1,  6081] loss: 0.6146175
[1,  6401] loss: 0.6561692
[1,  6721] loss: 0.6373615
[1,  7041] loss: 0.6207815
[1,  7361] loss: 0.6103640
[1,  7681] loss: 0.6402642
[1,  8001] loss: 0.6514095
[1,  8321] loss: 0.6123548
[1,  8641] loss: 0.6341721
[1,  8961] loss: 0.6271838
[1,  9281] loss: 0.6134559
[1,  9601] loss: 0.6192672
[1,  9921] loss: 0.6035549
[1, 10241] loss: 0.6277377
[1, 10561] loss: 0.6197450
[1, 10881] loss: 0.5947810
[1, 11201] loss: 0.6086776
[1, 11521] loss: 0.6133608
[1, 11841] loss: 0.6254957
[1, 12161] lo

### Complete loop

In [15]:
feature_combinations = [
    ["socio-demographics"],
    ["health"],
    ["social"],
    ["neighbourhood"],
    # ["socio-demographics", "health", "social", "neighbourhood"]
    ["health", "social"],
    ["social", "neighbourhood"],
    ["socio-demographics", "health"]
]

CSV_raw = "../ML_social.csv"
for use_feature_categories in feature_combinations:
    for target_index in range(4):
        print(f"Training for {use_feature_categories} on target {target_index}")

        # loss_log = exp_one_target_one_arch(use_feature_categories=feature_combination, 
        #                 csv_path=CSV_raw,
        #                 target_index=0, minimal_length=4, 
        #                 arch="RNN", learning_rate=0.001, momentum=0.9,
        #                 batch_size=32, print_batch_size=32*10)

        loss_log = exp_one_target_one_arch(use_feature_categories=use_feature_categories, 
                        csv_path=CSV_raw,
                        target_index=0, minimal_length=4, 
                        arch="RNN", learning_rate=0.01, momentum=0.9,
                        batch_size=32, print_batch_size=32*300)

Training for ['socio-demographics'] on target 0


/home/forrest/repos/AgeML/RNN/../preprocessing.py:196: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(csv_file)
100%|██████████| 26178/26178 [00:28<00:00, 903.20it/s]


(4, 9) (4, 4)
[1, 19201] loss: 1.2617053
[1, 28801] loss: 0.6170763
[1, 38401] loss: 0.6478221
[2, 19201] loss: 1.2588460
[2, 28801] loss: 0.6167653
[2, 38401] loss: 0.6477276
[3, 19201] loss: 1.2581606
[3, 28801] loss: 0.6164374
[3, 38401] loss: 0.6471054
Training for ['socio-demographics'] on target 1


/home/forrest/repos/AgeML/RNN/../preprocessing.py:196: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(csv_file)
100%|██████████| 26178/26178 [00:29<00:00, 878.60it/s]


(4, 9) (4, 4)
[1, 19201] loss: 1.2596166
[1, 28801] loss: 0.6170870
[1, 38401] loss: 0.6476438
[2, 19201] loss: 1.2587885
[2, 28801] loss: 0.6168561
[2, 38401] loss: 0.6478462
[3, 19201] loss: 1.2568596
[3, 28801] loss: 0.6171430
[3, 38401] loss: 0.6472450
Training for ['socio-demographics'] on target 2


...

(4, 12) (4, 4)
[1, 19201] loss: 1.2576736
[1, 28801] loss: 0.6163637
[1, 38401] loss: 0.6466519
[2, 19201] loss: 1.2543784
[2, 28801] loss: 0.6157057
[2, 38401] loss: 0.6458892
[3, 19201] loss: 1.2521604
[3, 28801] loss: 0.6154121
[3, 38401] loss: 0.6454750
Training for ['socio-demographics', 'health'] on target 2


/home/forrest/repos/AgeML/RNN/../preprocessing.py:196: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(csv_file)
100%|██████████| 26178/26178 [00:27<00:00, 935.85it/s]


(4, 12) (4, 4)
[1, 19201] loss: 1.2583430
[1, 28801] loss: 0.6164195
[1, 38401] loss: 0.6456820
[2, 19201] loss: 1.2536461
[2, 28801] loss: 0.6156178
[2, 38401] loss: 0.6451516
[3, 19201] loss: 1.2522950
[3, 28801] loss: 0.6155739
[3, 38401] loss: 0.6453114
Training for ['socio-demographics', 'health'] on target 3


/home/forrest/repos/AgeML/RNN/../preprocessing.py:196: DtypeWarning: Columns (5,24,28,29,30,31,32,33,34,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(csv_file)
100%|██████████| 26178/26178 [00:29<00:00, 897.39it/s]


(4, 12) (4, 4)
[1, 19201] loss: 1.2565199
[1, 28801] loss: 0.6161600
[1, 38401] loss: 0.6466316
[2, 19201] loss: 1.2539732
[2, 28801] loss: 0.6159730
[2, 38401] loss: 0.6464110
[3, 19201] loss: 1.2533160
[3, 28801] loss: 0.6157348
[3, 38401] loss: 0.6460555
